In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator
import random
import numpy as np
import pandas as pd
from typing import List, Dict, Union
import yaml
from bisect import bisect_left
from tqdm import tqdm
import calendar
import os
import math
from PIL import Image
import cv2
# from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration, AutoConfig
import torch
from transformers import AutoProcessor, Pix2StructForConditionalGeneration
import gc
import warnings
warnings.filterwarnings("ignore")

2023-07-05 03:04:09.660162: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib
2023-07-05 03:04:09.660305: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/li

In [2]:
chart_type = "vertical_bar"
version = 11

ver1.1 : gtの小数点を丸め処理

In [3]:
save_dir = f"./../input/benetech-synthesis-4th/{chart_type}_images_ver{version}"
os.makedirs(save_dir, exist_ok=True)
processor = AutoProcessor.from_pretrained("google/deplot")

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [5]:
def round_numerical(value: Union[int, float, str]) -> Union[str, float]:
    if isinstance(value, float):
        value = str(value)

        if "." in value:
            integer, decimal = value.split(".")
        else:
            integer = value
            
        if abs(float(integer)) >= 1000:
            integer = integer[:3] + "0"*(len(integer)-3)
            return integer
        elif abs(float(integer)) < 1000 and abs(float(integer)) >= 100:
            return integer
        elif abs(float(integer)) < 100 and abs(float(integer)) >= 10:
            decimal = decimal[:1]
        elif abs(float(integer)) < 10 and abs(float(integer)) >= 1:
            decimal = decimal[:2]
        else:
            decimal = decimal[:4]

        value = integer + "." + decimal
        
    if  isinstance(value, int):
        value = str(value)
        if abs(int(value)) >= 1000:
            value = value[:3] + "0"*(len(value)-3)
            return value        
        
    return value

def round_float(value: Union[int, float, str]) -> Union[str, float]:
    if isinstance(value, float):
        value = str(value)

        if "." in value:
            integer, decimal = value.split(".")
            if abs(float(integer)) >= 100:
                return integer
            elif abs(float(integer)) < 100 and abs(float(integer)) >= 10:
                decimal = decimal[:1]
            elif abs(float(integer)) < 10 and abs(float(integer)) >= 1:
                decimal = decimal[:2]
            else:
                decimal = decimal[:4]

            value = integer + "." + decimal
    return value

In [6]:
def get_x_values(source, num):
    val = []
    r = np.random.rand()
    
    if source == "numerical":
        if r < 0.7:
            start = random.randint(0, 5)
            val = [start+i for i in range(num)]
        else:
            start = random.randint(0, 10)
            inter = random.randint(1, 10)
            val = [start+(i*inter) for i in range(num)]
        return [str(v) for v in val]
    
    #　年
    elif source == "year":
        start = random.randint(1990, 2010)
        val = [str(start+i) for i in range(num)]
        return val
    
    # 1年以内の月
    elif source == "month" and num <= 12:
        if r < 0.5:
            month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        else:
            month = list(calendar.month_name)[1:]
        for i in range(num):
            val.append(month[i])
        return val
    
    # 1年を超える月
    elif source == "month" and num > 12:
        month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        year = random.randint(1990, 2010)
        for i in range(num):
            val.append(str(year) + "-" + month[i%12])
            if (i+1) % 12 == 0:
                year += 1
        # randomで2行表示
        if np.random.rand() < 0.5 and num < 10:
            val = [v.replace("-", "\n") for v in val]
        return val
    
    # 月日
    elif source == "day":
        month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        mon = random.choice(month)
        for i in range(num):
            val.append(f"{mon}-{i+1}")
        # randomで2行表示
        if np.random.rand() < 0.5 and num < 10:
            val = [v.replace("-", "\n") for v in val]
        return val
    
    # その他txtファイルから
    else:
        with open(f"{source}", "r") as f:
            for line in f:
                val.append(line.strip())
        values = random.sample(val, num)
        # randomで2行表示
        if num < 8 and np.random.rand() < 0.5:
            values = [replace_first_space_with_newline(v) for v in values]
            # randomで3行表示
            if np.random.rand() < 0.5:
                values = [replace_first_space_with_newline(v) for v in values]
        return values

In [7]:
def get_y_values(num):
    
    y_list = []
    r = np.random.rand()

    # 0固定 ~ 0.1
    if r < 0.1:
        a = 0
        b = 0.09
        
    # 0固定 ~ 1
    elif r < 0.2:
        a = 0
        b = 0.9
        
    # 0固定 ~ 10小数
    elif r < 0.25:
        a = 0
        b = random.uniform(0, 10)
        
    # 0固定 ~ 10整数
    elif r < 0.3:
        a = 0
        b = random.randint(1, 10)
        
    # 0固定 ~ 100 整数
    elif r < 0.4:
        a = 0
        b = random.randint(1, 100)
        
    # 0 ~ 100 整数
    elif r < 0.5:
        a = random.randint(0, 100)
        b = random.randint(a, 100)
        
    # 0固定 ~ 10000整数
    elif r < 0.6:
        a = 0
        b = random.randint(0, 1000)
        
    # 0 ~ 10000 整数
    elif r < 0.7:
        a = random.randint(0, 1000)
        b = random.randint(a, 1000)
        
    # 0固定 ~ 10000000
    elif r < 0.8:
        a = 0
        b = random.randint(0, 10000)
        
    # 0 ~ 10000000 整数
    elif r < 0.9:
        a = random.randint(0, 100000)
        b = random.randint(a, 100000)
        
    # 100000~ 10000000 整数
    else:
        a = random.randint(10000, 100000)
        b = random.randint(a, 100000)        
        
    for _ in range(num):
        if "." in str(b):
            y = random.uniform(a, b)
        else:
            y = random.randint(a, b)
        y_list.append(y)
    
    return y_list

In [8]:
def get_x_label(source):
    
    if source == "None":
        return ""
    
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [9]:
def get_y_label(source):
    
    if source == "None":
        return ""
    
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [10]:
def get_title(source):
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [11]:
def get_font(source):
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [12]:
def comma_format_func(value, tick_number):
    return f'{int(value):,}'

def dollar_format_func(value, tick_number):
    return f'${int(value):,}'

def yen_format_func(value, tick_number):
    return f'¥{int(value):,}'

def euro_format_func(value, tick_number):
    return f'€{int(value):,}'

def percent_format_func(value, tick_number):
    value = (value*10000)//1/10000
    if type(value) == float:
        return f'{value:.2%}'
    else:
        return f'{value}%'
    
def decimal_format_v1_func(value, tick_number):
    value = (value*10000)//1/10000
    return f'{value:.4}'
def decimal_format_v2_func(value, tick_number):
    value = (value*10000)//1/10000
    return f'{value:.4}0'

def zero_point_format_func(value, tick_number):
    value = (value*10000)//1/10000
    value = str(value)
    if value[0] == "0":
        value = value[1:]
    return value

In [13]:
def color_assignment(data, num):
    rand = np.random.rand()
    color1 = random.choices(data["colors"]["value"], k=1, weights=data["colors"]["weight"])[0]
    color2 = random.choices(data["colors"]["value"], k=1, weights=data["colors"]["weight"])[0]
    color3 = random.choices(data["colors"]["value"], k=1, weights=data["colors"]["weight"])[0]
    color4 = random.choices(data["colors"]["value"], k=1, weights=data["colors"]["weight"])[0]
    colors = []
    # 2色
    if rand < 0.1:
        p1 = num//2
        for i in range(num):
            if i < p1:
                colors.append(color1)
            else:
                colors.append(color2)
    # 3色
    elif rand < 0.2:
        p1 = num//3
        p2 = (num//3)*2
        for i in range(num):
            if i < p1:
                colors.append(color1)
            elif i < p2:
                colors.append(color2)
            else:
                colors.append(color3)
    # 4色
    elif rand < 0.3:
        p1 = num//4
        p2 = num//2
        p3 = (num//4)*3
        for i in range(num):
            if i < p1:
                colors.append(color1)
            elif i < p2:
                colors.append(color2)
            elif i < p3:
                colors.append(color3)
            else:
                colors.append(color4)
    # random
    elif rand < 0.4:
        for i in range(num):
            color = random.choice([color1, color2, color3, color4])
            colors.append(color)
    else:
        colors = [color1] * num
    
    return colors

In [14]:
def hatch_assignment(data, num):
    rand = np.random.rand()
    hatch1 = random.choices(data["hatching_h"]["value"], k=1, weights=data["hatching_h"]["weight"])[0]
    hatch2 = random.choices(data["hatching_h"]["value"], k=1, weights=data["hatching_h"]["weight"])[0]
    hatch3 = random.choices(data["hatching_h"]["value"], k=1, weights=data["hatching_h"]["weight"])[0]
    hatch4 = random.choices(data["hatching_h"]["value"], k=1, weights=data["hatching_h"]["weight"])[0]
    hatches = []
    # 2色
    if rand < 0.1:
        p1 = num//2
        for i in range(num):
            if i < p1:
                hatches.append(hatch1)
            else:
                hatches.append(hatch2)
    # 3色
    elif rand < 0.2:
        p1 = num//3
        p2 = (num//3)*2
        for i in range(num):
            if i < p1:
                hatches.append(hatch1)
            elif i < p2:
                hatches.append(hatch2)
            else:
                hatches.append(hatch3)
    # 4色
    elif rand < 0.3:
        p1 = num//4
        p2 = num//2
        p3 = (num//4)*3
        for i in range(num):
            if i < p1:
                hatches.append(hatch1)
            elif i < p2:
                hatches.append(hatch2)
            elif i < p3:
                hatches.append(hatch3)
            else:
                hatches.append(hatch4)
    # random
    elif rand < 0.4:
        for i in range(num):
            hatch = random.choice([hatch1, hatch2, hatch3, hatch4])
            hatches.append(hatch)
    else:
        hatches = [hatch1] * num
    
    return hatches

In [15]:
def replace_first_space_with_newline(text):
    index = text.find(" ")  # 最初の半角スペースのインデックスを検索
    if index != -1:
        text = text[:index] + "\n" + text[index+1:]  # 半角スペースを改行文字に置き換える
    return text

In [16]:
with open('graph_generation_ver4.yaml') as file:
    data_gen = yaml.safe_load(file)

In [17]:
def generate_graph(data, idx):
    # データ点数
    # 多め
    if np.random.rand() < 0.8:
        num_points = random.randint(3, 35)
    #少なめ
    else:
        num_points = random.randint(3, 8)
    
    # データ作成
    x_label_source = random.choices(data["categorical_label"]["value"], k=1, weights=data["categorical_label"]["weight"])[0]
    x_label = get_x_label(x_label_source)
    #print(x_label)
    y_label_source = random.choices(data["numerical_label"]["value"], k=1, weights=data["numerical_label"]["weight"])[0]
    y_label = get_y_label(y_label_source)
    #print(y_label)
    x_label_source = random.choices(data["categorical_value"]["value"], k=1, weights=data["categorical_value"]["weight"])[0]
    x_values = get_x_values(x_label_source, num_points)
    #print(x_values)
    y_values = get_y_values(num_points)
    #print(y_values)
    
    # x_labelの調整
    if x_label_source == "year":
        x_label = "Year"
    elif x_label_source == "month":
        x_label = "Month"
    elif x_label_source == "day":
        x_label = "Day"
    elif x_label_source == "country.txt":
        x_label = random.choice(["Counties", "Country", "Region" ,"Nation", "Area", "Place"])
    elif x_label_source == "region.txt":
        x_label = random.choice(["Region" ,"Nation", "Area", "Place"])
    elif x_label_source == "numerical":
        x_label_source = random.choice(["numericals.txt" ,"measurements.txt"])
        x_label = get_y_label(y_label_source)
    
    # 大文字・小文字・キャピタライズ
    x_rand = np.random.rand()
    if x_rand < 0.3:
        x_values = [x.upper() for x in x_values]
    elif x_rand < 0.7:
        x_values = [x.lower() for x in x_values]
    else:
        x_values = [x.lower().capitalize() for x in x_values]
    
    
    # フォントとフォントサイズの設定
    fontsize = random.choices(data["fontsize"]["value"], k=1, weights=data["fontsize"]["weight"])[0]
    if num_points > 15:
        fontsize = 7
    elif num_points > 20:
        fontsize = 6
    elif num_points > 27:
        fontsize = 5
    plt.rcParams['font.size'] = fontsize
    
    font_source = random.choices(data["font"]["value"], k=1, weights=data["font"]["weight"])[0]
    font = get_font(font_source)
    #print(font)
        
    # グラフのサイズを指定 例：4.56=456 pixel
    size1 = random.randint(400, 800)
    size2 = random.randint(300, size1)
    size1 /= 100
    size2 /= 100
    
    facecolor =  random.choices(data["facecolor"]["value"], k=1, weights=data["facecolor"]["weight"])[0]
    
    fig, ax = plt.subplots(figsize=(size1, size2))
    fig.set_facecolor(facecolor)
    
    color = random.choices(data["colors"]["value"], k=1, weights=data["colors"]["weight"])[0]
    edgecolor = random.choices(data["colors"]["value"], k=1, weights=data["colors"]["weight"])[0]
    #linewidth = random.choices(data["linewidth"]["value"], k=1, weights=data["linewidth"]["weight"])[0]
    barwidth = random.choices(data["barwidth"]["value"], k=1, weights=data["barwidth"]["weight"])[0]
    background_color = random.choices(data["background_color"]["value"], k=1, weights=data["background_color"]["weight"])[0]
    
    # カラーバリエーションを追加
    colors = color_assignment(data, num_points)
    
    # エラーバーの設定
    rand = np.random.rand()
    # 変動
    if rand < 0.075:
        errors = [y*random.uniform(0, 0.2) for y in y_values]
        capsize= random.randint(0, 3)
    # 固定
    elif rand < 0.15:
        error = max(y_values)*random.uniform(0, 0.2)
        errors = [error for _ in y_values]
        capsize= random.randint(0, 3)
    # なし
    else:
        errors = [0 for _ in y_values]
        capsize = 0
    
    # hatching
    if rand < 0.2 and len(set(colors)) == 1:
        hatch_patterns = hatch_assignment(data, num_points)
        bars = ax.bar(x_values, y_values, width=barwidth, color=color, edgecolor=edgecolor, linewidth=0, yerr=errors, capsize=capsize)
        # 各バーに対してハッチングパターンを設定
        for bar, pattern in zip(bars, hatch_patterns):
            bar.set_hatch(pattern)
    # 枠線のみ、塗り潰しなし
    elif rand < 0.25:
        linewidth = 1
        ax.bar(x_values, y_values, width=barwidth, color=background_color, edgecolor=edgecolor, linewidth=linewidth, yerr=errors, capsize=capsize)
    else:
        ax.bar(x_values, y_values, width=barwidth, color=colors, edgecolor=edgecolor, linewidth=0, yerr=errors, capsize=capsize)
    
    # major tickの設定
    tick_direction = random.choices(data["tick_direction"]["value"], k=1, weights=data["tick_direction"]["weight"])[0]
    # 削除
    if np.random.rand() < 0.3:
        plt.tick_params(axis='x', which='major', length=0)
    else:
        plt.tick_params(axis='x', which='major', direction=tick_direction)
        
    # 上部と右側の枠線を非表示にする
    if rand < 0.2:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    # 合計文字数で回転を設定
    # 横配置
    # 改行文字で区切ったときの1行の最大文字数
    num_str = [max([len(xi) for xi in str(x).split("\n")]) for x in x_values]
    if sum(num_str) < 40 and max(num_str) < 9:
        ax.set_xticklabels(x_values, rotation=0, fontname=font)
        # tickをbar間に設定
        if np.random.rand() < 0.3:
            plt.tick_params(axis='x', which='major', length=0)
            minor_locator = MultipleLocator(0.5)
            plt.gca().xaxis.set_minor_locator(minor_locator)
            plt.gca().tick_params(axis='x', which='minor', direction=tick_direction)
    # 縦配置
    elif num_points > 15:
        # 90 degree
        if np.random.rand() < 0.8:
            ax.set_xticklabels(x_values, rotation=90, fontname=font)
        # 270 degree
        else:
            ax.set_xticklabels(x_values, rotation=270, fontname=font)
    # 斜め配置
    else:
        # 45 degree
        if np.random.rand() < 0.8:
            ax.set_xticklabels(x_values, rotation=45, ha="right", fontname=font)
        # 315 degree
        else:
            ax.set_xticklabels(x_values, rotation=315, ha="left", fontname=font)
        # tickをbar間に設定
        if np.random.rand() < 0.3:
            plt.tick_params(axis='x', which='major', length=0)
            minor_locator = MultipleLocator(0.5)
            plt.gca().xaxis.set_minor_locator(minor_locator)
            plt.gca().tick_params(axis='x', which='minor', direction=tick_direction)
    
    #  y-tickにフォント設定
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)
        
    # y軸のformat（カンマ区切り、¥, $つけるなど）
    if min(y_values) > 1000:
        y_rand = np.random.rand()
        if y_rand < 0.25:
            ax.yaxis.set_major_formatter(ticker.FuncFormatter(comma_format_func))
        elif y_rand < 0.4:
            ax.yaxis.set_major_formatter(ticker.FuncFormatter(dollar_format_func))
        elif y_rand < 0.45:
            ax.yaxis.set_major_formatter(ticker.FuncFormatter(yen_format_func))
        elif y_rand < 0.5:
            ax.yaxis.set_major_formatter(ticker.FuncFormatter(euro_format_func))
    elif (min(y_values) >= 0 and max(y_values) < 100):
        y_rand = np.random.rand()
        # %マークを末尾に追加
        if y_rand < 0.2:    
            ax.yaxis.set_major_formatter(ticker.FuncFormatter(percent_format_func))
        # 最初の0を削除
        elif y_rand < 0.3 and max(y_values) < 1:
            ax.yaxis.set_major_formatter(ticker.FuncFormatter(zero_point_format_func))
        # 小数点第n位まで表示
        elif (y_rand < 0.7) & (max(y_values) < 10 and max(y_values) > 0.1):
            if np.random.rand() < 0.5:
                ax.yaxis.set_major_formatter(ticker.FuncFormatter(decimal_format_v1_func))
            else:
                ax.yaxis.set_major_formatter(ticker.FuncFormatter(decimal_format_v2_func))
    
    
    # ラベルにフォント設定
    if x_label:
        ax.set_xlabel(x_label, fontname=font, labelpad=20)
    if y_label:
        ax.set_ylabel(y_label, fontname=font, labelpad=20)
    
    # グラフタイトル
    title_source = random.choices(data["title"]["value"], k=1, weights=data["title"]["weight"])[0]
    if title_source != "None":
        title = get_title(title_source)
        ax.set_title(title, fontname=font)
    else:
        title = ""
    
    # 背景色
    if color != "k":
        ax.set_facecolor(background_color)
        
    # grid線
    grid = random.choices(data["grid"]["value"], k=1, weights=data["grid"]["weight"])[0]
    # vertical barではy-grid優先
    if grid == "x" and np.random.rand() < 0.8:
        grid = "y"
    if grid != "None":
        grid_color = background_color = random.choices(data["background_color"]["value"], k=1, weights=data["background_color"]["weight"])[0]
        grid_linestyle = random.choices(data["linestyle"]["value"], k=1, weights=data["linestyle"]["weight"])[0]
        if grid == "bose":
            ax.grid(color=grid_color, linestyle=grid_linestyle)
        else:
            ax.grid(color=grid_color, linestyle=grid_linestyle, axis=grid)
            
    # barの上に数字を表示
    if max(y_values) < 100 and type(max(y_values)) == int and num_points < 20 and sum(errors) == 0:
        if np.random.rand() < 0.4:
            for i, value in enumerate(y_values):
                plt.text(i, value, str(value), ha='center', va='bottom')
        
        
    # 画像の保存
    id = str(idx).zfill(8)
    path = os.path.join(save_dir, f"{id}.jpg")
    fig.savefig(path, bbox_inches='tight')
    
    # 画質を落とす
    image = Image.open(path)
    quality = random.randint(5, 100)
    # greyscaleで保存
    if np.random.rand() < 0.05:
        image = image.convert("L")
        image = image.convert("RGB")
    image.save(path, quality=quality)    
    
    # -------------------- gt作成 --------------------
    
    y_values = [round_float(y) for y in y_values]
    x_values = [str(x).replace("\n", " ") for x in x_values]
        
    # gt文字列を作成
    gt_string = []
    for x, y in zip(x_values, y_values):
        gt_string.append(f"{x} | {y}")
    gt_string = "<0x0A>".join(gt_string)
    gt_string += processor.tokenizer.eos_token
    
    # if idx % 1000 == 0:
    #     print(gt_string)
    #     plt.show()
    #     #print(font)
    
    plt.close()
    #plt.show()
    
    df = pd.DataFrame({"id": idx, "ground_truth": gt_string, "fold": -2, "x": [x_values], "y": [y_values], "source": "synthesis", "chart-type": chart_type, "image_path":path[3:]})
    gc.collect()
    return df

In [18]:
seed_everything(42)
from multiprocessing import Pool
import multiprocessing
num_cores = multiprocessing.cpu_count()
def _generate_graph(idx):
    return generate_graph(data_gen,idx)

num = 40000
args = list(range(num))
run = _generate_graph

with Pool(processes=num_cores) as pool:
    imap = pool.imap_unordered(run,args)
    df_list = list(tqdm(imap, total=len(args)))
df = pd.concat(df_list).reset_index(drop=True)
df.to_csv(f"./../input/benetech-synthesis-4th/graph_synthesis_{chart_type}_{num}_ver{version}.csv", index=False)

100%|██████████| 40000/40000 [41:08<00:00, 16.21it/s]
